# Use basic probability to identify anomalous requests. Using the methods covered in this lesson, examine the rest of the features in the api access logs data set.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env

In [2]:
url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)
df.head()

,entry
0,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
1,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
2,97.105.19.58 - - [16/Apr/2019:19:34:44 +0000] ...
3,97.105.19.58 - - [16/Apr/2019:19:34:46 +0000] ...
4,97.105.19.58 - - [16/Apr/2019:19:34:48 +0000] ...


In [3]:
df.entry[0]

'97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] "GET /api/v1/sales?page=81 HTTP/1.1" 200 512495 "-" "python-requests/2.21.0"'

In [4]:
df.entry[0].split()

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [5]:
#function to create a organized dataframe
def log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)
    

In [6]:
df = df.entry.apply(log_entry)
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [7]:
# convert timestamp to index of the dataframe
df.timestamp = pd.to_datetime(df.timestamp)
df = df.set_index('timestamp')


In [8]:
df.head(2)

,ip,request_method,request_path,http_version,status_code,size,user_agent
timestamp,,,,,,,
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0


# detecting anomalies
- status code

In [ ]:
df.status_code.value_counts()

In [9]:
# request_path value counts
df.request_path.value_counts()

/api/v1/sales?page=2                 709
/api/v1/items                        464
/api/v1/items?page=2                 291
/api/v1/items?page=3                 219
/api/v1/stores                       162
                                    ... 
/api/v1/items?page=0                   1
/api/v1/stores?page=999                1
/api/v1/stores?page=2                  1
/api/v1/stores?page=666                1
/api/v1/items/api/v1/items?page=3      1
Name: request_path, Length: 218, dtype: int64

In [10]:
# converting the request path to dataframe
request_path_df = pd.DataFrame(df.request_path.value_counts(dropna= False)).reset_index().\
                    rename(columns={'index': 'request_path', 'request_path': 'request_path_cnt'})

In [11]:
request_path_df.head()

,request_path,request_path_cnt
0,/api/v1/sales?page=2,709
1,/api/v1/items,464
2,/api/v1/items?page=2,291
3,/api/v1/items?page=3,219
4,/api/v1/stores,162


In [12]:
# calculate the probability for each request path
# probability= count of path/ total count of paths
request_path_df2 = pd.DataFrame((df.request_path.value_counts(dropna= False))/ df.request_path.count()).reset_index().\
                    rename(columns={'index':'request_path', 'request_path':'probability'})

In [14]:
request_path_df2

,request_path,probability
0,/api/v1/sales?page=2,0.050737
1,/api/v1/items,0.033205
2,/api/v1/items?page=2,0.020824
3,/api/v1/items?page=3,0.015672
4,/api/v1/stores,0.011593
...,...,...
213,/api/v1/items?page=0,0.000072
214,/api/v1/stores?page=999,0.000072
215,/api/v1/stores?page=2,0.000072
216,/api/v1/stores?page=666,0.000072


In [15]:
request_path_df= request_path_df.merge(request_path_df2,on='request_path')
request_path_df.shape

(218, 3)

In [16]:
request_path_df.head()

,request_path,request_path_cnt,probability
0,/api/v1/sales?page=2,709,0.050737
1,/api/v1/items,464,0.033205
2,/api/v1/items?page=2,291,0.020824
3,/api/v1/items?page=3,219,0.015672
4,/api/v1/stores,162,0.011593


In [ ]:
request_path_df[request_path_df.probability < 0.01]

97.105.19.58      11998
173.173.113.51     1059
72.181.113.170      613
72.181.105.81       246
24.26.242.9          21
68.201.219.223       21
70.121.214.34         2
52.87.230.102         2
35.175.171.137        2
54.145.52.184         1
3.92.201.136          1
35.174.209.2          1
34.229.70.250         1
54.172.14.223         1
34.207.64.242         1
3.88.129.158          1
52.91.30.150          1
52.90.165.200         1
45.23.250.16          1
Name: ip, dtype: int64